<a href="https://colab.research.google.com/github/yukinaga/minnano_rl/blob/main/section_4/02_deep_reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深層強化学習の実装
重力下で飛行する物体の制御を、深層強化学習により行います。  
深層強化学習では、Q-Tableの代わりにニューラルネットワークを使用します。  
ニューラルネットワークを実装するためのフレームワークとして、PyTorchを使用します。

## ライブラリの導入
数値計算のためにNumPy、グラフ表示のためにmatplotlib、ニューラルネットワークを実装するためのフレームワークとしてPyTorchを導入します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

## Netクラス
`nn.Module`モジュールを継承したクラスとして、ニューラルネットワークを実装します。

In [ ]:
class Net(nn.Module):
    def __init__(self, n_state, n_mid, n_action):
        super().__init__()
        self.fc1 = nn.Linear(n_state, n_mid)  # 全結合層
        self.fc2 = nn.Linear(n_mid, n_mid)
        self.fc3 = nn.Linear(n_mid, n_action)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Brainクラス
エージェントの頭脳となるクラスです。Q値を出力するニューラルネットワークを構築し、Q値が正解に近づくように訓練します。  
Q学習に用いる式は以下の通りです。  

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \eta\left(R_{t+1}+\gamma \max_{a}Q(s_{t+1}, a) - Q(s_{t}, a_{t})\right) $$

ここで、$a_{t}$は行動、$s_t$は状態、$Q(s_t,a_t) $はQ値、$\eta$は学習係数、$R_{t+1}$は報酬、$\gamma$は割引率になります。  
次の状態における最大のQ値を使用するのですが、ディープラーニングの正解として用いるのは上記の式のうちの以下の部分です。  

$$R_{t+1}+\gamma \max_{a}Q(s_{t+1}, a_{t})$$

以下の`Brain`クラスにおけるtrainメソッドでは、正解として上記を用います。  
また、ある状態における行動を決定する`get_action`メソッドでは、ε-greedy法により行動が選択されます。

In [ ]:
class Brain:
    def __init__(self, n_state, n_action, net, loss_fnc, optimizer, is_gpu, gamma=0.9, r=0.99, lr=0.01):
        self.n_state = n_state  # 状態の数
        self.n_action = n_action  # 行動の数

        self.net = net  # ニューラルネットワークのモデル
        self.loss_fnc = loss_fnc  # 誤差関数
        self.optimizer = optimizer  # 最適化アルゴリズム
        self.is_gpu = is_gpu  # GPUを使うかどうか
        if self.is_gpu:
            self.net.cuda()  # GPU対応

        self.eps = 1.0  # ε
        self.gamma = gamma  # 割引率
        self.r = r  # εの減衰率
        self.lr = lr  # 学習係数

    def train(self, states, next_states, action, reward, terminal):  # ニューラルネットワークを訓練

        states = torch.from_numpy(states).float()
        next_states = torch.from_numpy(next_states).float()
        if self.is_gpu:
            states, next_states = states.cuda(), next_states.cuda()  # GPU対応
            
        self.net.eval()  # 評価モード
        next_q = self.net.forward(next_states)
        self.net.train()  # 訓練モード
        q = self.net.forward(states)

        t = q.clone().detach()
        if terminal:
            t[:, action] = reward  #  エピソード終了時の正解は、報酬のみ
        else:
            t[:, action] = reward + self.gamma*np.max(next_q.detach().cpu().numpy(), axis=1)[0]
            
        loss = self.loss_fnc(q, t)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def get_action(self, states):  # 行動を取得
        states = torch.from_numpy(states).float()
        if self.is_gpu:
            states = states.cuda()  # GPU対応

        if np.random.rand() < self.eps:  # ランダムな行動
            action = np.random.randint(self.n_action)
        else:  # Q値の高い行動を選択
            q = self.net.forward(states)
            action = np.argmax(q.detach().cpu().numpy(), axis=1)[0]
        if self.eps > 0.1:  # εの下限
            self.eps *= self.r
        return action

## エージェントのクラス
エージェントをクラスとして実装します。  
x座標が-1から1まで、y座標が-1から1までの正方形の領域を考えますが、エージェントの初期位置は左端中央とします。  
そして、エージェントが右端に達した際は報酬として1を与え、終了とします。  
また、エージェントが上端もしくは下端に達した際は報酬として-1を与え、終了とします。  
  
x軸方向には等速度で移動します。  
行動には、自由落下とジャンプの2種類があります。自由落下の場合は重量加速度をy速度に加えます。ジャンプの場合は、y速度を予め設定した値に変更します。

In [ ]:
class Agent:
    def __init__(self, v_x, v_y_sigma, v_jump, brain):
        self.v_x = v_x  # x速度
        self.v_y_sigma = v_y_sigma  # y速度、初期値の標準偏差
        self.v_jump = v_jump  # ジャンプ速度
        self.brain = brain
        self.reset()

    def reset(self):
        self.x = -1  # 初期x座標
        self.y = 0  # 初期y座標
        self.v_y = self.v_y_sigma * np.random.randn()  # 初期y速度

    def step(self, g):  # 時間を1つ進める g:重力加速度
        states = np.array([[self.y, self.v_y]])
        self.x += self.v_x
        self.y += self.v_y

        reward = 0  # 報酬
        terminal = False  # 終了判定
        if self.x>1.0:
            reward = 1
            terminal = True
        elif self.y<-1.0 or self.y>1.0:
            reward = -1
            terminal = True

        action = self.brain.get_action(states)
        if action == 0:
            self.v_y -= g   # 自由落下
        else:
            self.v_y = self.v_jump  # ジャンプ
        next_states = np.array([[self.y, self.v_y]])
        self.brain.train(states, next_states, action, reward, terminal)

        if terminal:
            self.reset()

## 環境のクラス
環境をクラスとして実装します。  
このクラスの役割は、重力加速度を設定し、時間を前に進めるのみです。

In [ ]:
class Environment:
    def __init__(self, agent, g):
        self.agent = agent
        self.g = g  # 重力加速度

    def step(self):
        self.agent.step(self.g)
        return (self.agent.x, self.agent.y)

## アニメーション
今回は、matplotlibを使ってエージェントの飛行をアニメーションで表します。  
アニメーションには、matplotlib.animationのFuncAnimation関数を使用します。  

In [ ]:
def animate(environment, interval, frames):
    fig, ax = plt.subplots()
    plt.close()
    ax.set_xlim(( -1, 1))
    ax.set_ylim((-1, 1))
    sc = ax.scatter([], [])

    def plot(data):
        x, y = environment.step()
        sc.set_offsets(np.array([[x, y]]))
        return (sc,)

    return animation.FuncAnimation(fig, plot, interval=interval, frames=frames, blit=True)

## 深層強化学習の実行
ニューラルネットワーク、Brain、エージェント、環境の設定を行い、深層強化学習を実行します。

In [ ]:
n_state = 2
n_mid = 32
n_action = 2

net = Net(n_state, n_mid, n_action)

loss_fnc = nn.MSELoss()  # 誤差関数
optimizer = optim.RMSprop(net.parameters(), lr=0.01)  # 最適化アルゴリズム
is_gpu = True

brain = Brain(n_state, n_action, net, loss_fnc, optimizer, is_gpu)

v_x = 0.05
v_y_sigma = 0.1
v_jump = 0.2
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2
environment = Environment(agent, g)

anim = animate(environment, 50, 1024)
rc("animation", html="jshtml")
anim

学習が進むにつれて、エージェントは状態に応じて適切な行動を選択できるようになります。